In [384]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Cleaning the Data

We label encode the different warehouses and remove entries where the dates doesn't exist. The date is important because we're attempting a time series analysis. We also transform the 'Date' column into a datetime datatype since its easier to work with.

In [433]:
hist_prod_dem = pd.read_csv('../data/HistProdDem.csv')
hist_prod_dem['Date'] = pd.to_datetime(hist_prod_dem['Date'])
hist_prod_dem['Order_Demand'] = hist_prod_dem['Order_Demand'].apply(lambda x: x.strip('()'))
hist_prod_dem['Order_Demand'] = hist_prod_dem['Order_Demand'].astype(int)

le = LabelEncoder()
le.fit(hist_prod_dem['Warehouse'])

warehouse_le = pd.DataFrame(le.transform(hist_prod_dem['Warehouse']), columns=['Warehouse_cat'])
hist_prod_dem = hist_prod_dem.join(warehouse_le)

has_dates = hist_prod_dem[hist_prod_dem['Date'].notnull()]

# Choosing Products to Analyze

To decide which products to analyze, we simply look at all the products and find the ones with the most entries in the table. We're hoping that more entries means less gaps to fill in in our time series.  

In [489]:
prods_nums = []
for prod in has_dates.Product_Code.unique():
    entries = len(has_dates[has_dates['Product_Code'] == prod])
    prods_nums.append((prod, entries))
    
prods_nums.sort(key=lambda x: x[1])
prods_nums.reverse()
prods_nums

[('Product_1359', 16936),
 ('Product_1295', 10575),
 ('Product_1378', 9770),
 ('Product_0620', 9428),
 ('Product_1286', 8888),
 ('Product_1382', 8619),
 ('Product_1453', 8503),
 ('Product_1294', 8384),
 ('Product_1377', 7872),
 ('Product_1287', 7829),
 ('Product_1274', 7048),
 ('Product_1016', 6571),
 ('Product_1248', 6348),
 ('Product_0613', 6079),
 ('Product_1241', 5476),
 ('Product_1456', 5442),
 ('Product_1470', 5084),
 ('Product_0031', 5013),
 ('Product_1410', 4828),
 ('Product_0608', 4803),
 ('Product_1360', 4801),
 ('Product_1451', 4703),
 ('Product_1250', 4622),
 ('Product_1381', 4583),
 ('Product_1264', 4519),
 ('Product_0033', 4479),
 ('Product_1262', 4478),
 ('Product_1452', 4462),
 ('Product_1023', 4381),
 ('Product_1463', 4331),
 ('Product_1512', 4233),
 ('Product_1886', 4229),
 ('Product_0349', 4116),
 ('Product_0408', 4101),
 ('Product_1496', 4075),
 ('Product_1350', 3906),
 ('Product_0979', 3889),
 ('Product_0407', 3791),
 ('Product_1430', 3778),
 ('Product_2167', 3642)

We grab the top four as our datasets to work with. 


In [490]:
prod1359 = has_dates[has_dates['Product_Code'] == 'Product_1359']
prod1295 = has_dates[has_dates['Product_Code'] == 'Product_1295']
prod1378 = has_dates[has_dates['Product_Code'] == 'Product_1378']
prod0620 = has_dates[has_dates['Product_Code'] == 'Product_0620']

# Feature Engineering

We split the date into months, years, week in the year, and quarter because these are possible windows over which to do our analysis. We also notice that in 3 of our datasets, only one warehouse is used, which simplifies our next step. We add the previous week's demand as a feature to each line, filling in the initial line with 0. This process is simple for the three sets where the warehouse is the same for all weeks. For the fourth, we need to first split it into 3 sets based on which warehouse the demand came from before doing the same as the previous 3. There's a problem with this approach in that not all weeks have a demand and some warehouses have demand when others don't. 

In [436]:
prod1359 = prod1359.join(pd.Series(prod1359.loc[:,'Date'].dt.weekday, name='Weekday')).join(pd.Series(prod1359.loc[:,'Date'].dt.isocalendar().week, name='Week')).join(pd.Series(prod1359.loc[:,'Date'].dt.month, name='Month')).join(pd.Series(prod1359.loc[:,'Date'].dt.quarter, name='Quarter')).join(pd.Series(prod1359.loc[:,'Date'].dt.year, name='Year'))
prod1295 = prod1295.join(pd.Series(prod1295.loc[:,'Date'].dt.weekday, name='Weekday')).join(pd.Series(prod1295.loc[:,'Date'].dt.isocalendar().week, name='Week')).join(pd.Series(prod1295.loc[:,'Date'].dt.month, name='Month')).join(pd.Series(prod1295.loc[:,'Date'].dt.quarter, name='Quarter')).join(pd.Series(prod1295.loc[:,'Date'].dt.year, name='Year'))
prod1378 = prod1378.join(pd.Series(prod1378.loc[:,'Date'].dt.weekday, name='Weekday')).join(pd.Series(prod1378.loc[:,'Date'].dt.isocalendar().week, name='Week')).join(pd.Series(prod1378.loc[:,'Date'].dt.month, name='Month')).join(pd.Series(prod1378.loc[:,'Date'].dt.quarter, name='Quarter')).join(pd.Series(prod1378.loc[:,'Date'].dt.year, name='Year'))
prod0620 = prod0620.join(pd.Series(prod0620.loc[:,'Date'].dt.weekday, name='Weekday')).join(pd.Series(prod0620.loc[:,'Date'].dt.isocalendar().week, name='Week')).join(pd.Series(prod0620.loc[:,'Date'].dt.month, name='Month')).join(pd.Series(prod0620.loc[:,'Date'].dt.quarter, name='Quarter')).join(pd.Series(prod0620.loc[:,'Date'].dt.year, name='Year'))

In [437]:
prod1359 = prod1359.sort_values(by='Date').reset_index().drop('index', axis=1)
prod1295 = prod1295.sort_values(by='Date').reset_index().drop('index', axis=1)
prod1378 = prod1378.sort_values(by='Date').reset_index().drop('index', axis=1)
prod0620 = prod0620.sort_values(by='Date').reset_index().drop('index', axis=1)

In [438]:
print(prod1378['Warehouse_cat'].unique()) # nice, there's only 1
print(prod1359['Warehouse_cat'].unique()) # also only 1
print(prod1295['Warehouse_cat'].unique()) # also only 1
print(prod0620['Warehouse_cat'].unique()) # tres

[2]
[2]
[2]
[2 1 3]


In [439]:
dates_prod1378 = prod1378[['Date', 'Order_Demand']].groupby(['Date'], as_index=False).agg('sum')
dates_prod1295 = prod1295[['Date', 'Order_Demand']].groupby(['Date'], as_index=False).agg('sum')
dates_prod1359 = prod1359[['Date', 'Order_Demand']].groupby(['Date'], as_index=False).agg('sum')
dates_prod0620 = prod0620[['Date', 'Warehouse_cat', 'Order_Demand']].groupby(['Date', 'Warehouse_cat'], as_index=False).agg('sum')

In [459]:
weeks_prod1378 = prod1378[['Week', 'Year', 'Order_Demand']].groupby(['Week', 'Year'], as_index=False).agg('sum').sort_values(by=['Year', 'Week']).reset_index().drop('index', axis=1)
weeks_prod1378['Previous_Week'] = weeks_prod1378['Order_Demand'].shift().fillna(0)

weeks_prod1359 = prod1359[['Week', 'Year', 'Order_Demand']].groupby(['Week', 'Year'], as_index=False).agg('sum').sort_values(by=['Year', 'Week']).reset_index().drop('index', axis=1)
weeks_prod1359['Previous_Week'] = weeks_prod1359['Order_Demand'].shift().fillna(0)

weeks_prod1295 = prod1295[['Week', 'Year', 'Order_Demand']].groupby(['Week', 'Year'], as_index=False).agg('sum').sort_values(by=['Year', 'Week']).reset_index().drop('index', axis=1)
weeks_prod1295['Previous_Week'] = weeks_prod1295['Order_Demand'].shift().fillna(0)

weeks_prod0620 = prod0620[['Week', 'Year', 'Warehouse_cat', 'Order_Demand']].groupby(['Week', 'Year', 'Warehouse_cat'], as_index=False).agg('sum').sort_values(by=['Year', 'Week']).reset_index().drop('index', axis=1)
#weeks_prod0620['Previous_Week'] = weeks_prod0620['Order_Demand'].shift().fillna(0)

In [464]:
weeks_prod0620_wh1 = pd.DataFrame(weeks_prod0620[weeks_prod0620['Warehouse_cat'] == 1])
weeks_prod0620_wh2 = pd.DataFrame(weeks_prod0620[weeks_prod0620['Warehouse_cat'] == 2])
weeks_prod0620_wh3 = pd.DataFrame(weeks_prod0620[weeks_prod0620['Warehouse_cat'] == 3])

weeks_prod0620_wh1['Previous_Week'] = weeks_prod0620_wh1['Order_Demand'].shift().fillna(0)
weeks_prod0620_wh2['Previous_Week'] = weeks_prod0620_wh2['Order_Demand'].shift().fillna(0)
weeks_prod0620_wh3['Previous_Week'] = weeks_prod0620_wh3['Order_Demand'].shift().fillna(0)

In [521]:
weeks_prod0620_wha = weeks_prod0620_wh1.merge(weeks_prod0620_wh2, on=['Year', 'Week'], how='outer', suffixes=['_1', '_2'])
weeks_prod0620_wha = weeks_prod0620_wha.merge(weeks_prod0620_wh3, on=['Year', 'Week'], how='outer', suffixes=['', '_3'])
weeks_prod0620_wha = weeks_prod0620_wha.sort_values(by=['Year', 'Week']).fillna(method='ffill').fillna(0).drop(['Warehouse_cat_1', 'Warehouse_cat', 'Warehouse_cat_2'], axis=1)

In [524]:
weeks_prod0620_wha['Previous_Week_3'] = weeks_prod0620_wha['Previous_Week']
weeks_prod0620_wha['Order_Demand_3'] = weeks_prod0620_wha['Order_Demand']

total_order_demand = weeks_prod0620_wha['Order_Demand_1'] + weeks_prod0620_wha['Order_Demand_2'] + weeks_prod0620_wha['Order_Demand_3']
weeks_prod0620_wha['Order_Demand'] = total_order_demand

weeks_prod0620 = weeks_prod0620_wha.drop(['Previous_Week', 'Order_Demand_1', 'Order_Demand_2', 'Order_Demand_3'], axis=1)

# Time Series Analysis

We finally get to our analyses. We use gradient boosting regression to create our model and validate using scikit-learns TimeSeriesSplit, which splits the data set into (approximately) each year between 2012 and 2016. Then each year is added to the training set one at a time with the next year used as the validation set. 

In [441]:
gbr = GradientBoostingRegressor()

In [442]:
tss = TimeSeriesSplit()

In [443]:
X = weeks_prod1378[['Week', 'Year', 'Previous_Week']].values
y = weeks_prod1378['Order_Demand'].values

for train_index, test_index in tss.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gbr.fit(X_train, y_train)
    print(mean_squared_error(gbr.predict(X_test), y_test))

11802687766.580246
9015970804.086788
15286885829.052425
23430455132.669075
12552458636.213755


In [444]:
X = weeks_prod1359[['Week', 'Year', 'Previous_Week']].values
y = weeks_prod1359['Order_Demand'].values

for train_index, test_index in tss.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gbr.fit(X_train, y_train)
    print(mean_squared_error(gbr.predict(X_test), y_test))

436722314220.6217
748885706752.4078
516263399779.5274
582220855106.6759
666398365667.7367


In [445]:
X = weeks_prod1295[['Week', 'Year', 'Previous_Week']].values
y = weeks_prod1295['Order_Demand'].values

for train_index, test_index in tss.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gbr.fit(X_train, y_train)
    print(mean_squared_error(gbr.predict(X_test), y_test))

27506966725.627316
24910112466.992756
32267651055.866405
37982157159.31326
38120673009.07312


In [525]:
X = weeks_prod0620[['Week', 'Year', 'Previous_Week_1', 'Previous_Week_2', 'Previous_Week_3']].values
y = weeks_prod0620['Order_Demand'].values

for train_index, test_index in tss.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gbr.fit(X_train, y_train)
    print(mean_squared_error(gbr.predict(X_test), y_test))

80425.41069493862
99100.77503514152
148717.12952099284
390946.086603644
73883.08551241468


In short, these models are all trash. There are a variety of ways we could try to improve them. For one, try to take advantage of information about other product's demand in relation to the target product. We could also try to fully fill out each week with a running average